In [1]:
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = "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"

In [2]:
%pip install -qU langchain-google-community\[gmail\]

Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_google_community import GmailToolkit

toolkit = GmailToolkit()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=560333382753-32umaapbb1u5u0hhkmj1h5s38l7b4v4r.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A57528%2F&scope=https%3A%2F%2Fmail.google.com%2F&state=5zO39b39KgtvbHOha2bwR9auujyYXh&access_type=offline


In [6]:
from langchain_google_community.gmail.utils import (
    build_resource_service,
    get_gmail_credentials,
)

# Can review scopes here https://developers.google.com/gmail/api/auth/scopes
# For instance, readonly scope is 'https://www.googleapis.com/auth/gmail.readonly'
credentials = get_gmail_credentials(
    token_file="token.json",
    scopes=["https://mail.google.com/"],
    client_secrets_file="credentials.json",
)
api_resource = build_resource_service(credentials=credentials)
toolkit = GmailToolkit(api_resource=api_resource)

In [7]:
tools = toolkit.get_tools()
tools

[GmailCreateDraft(api_resource=<googleapiclient.discovery.Resource object at 0x15df74aa0>),
 GmailSendMessage(api_resource=<googleapiclient.discovery.Resource object at 0x15df74aa0>),
 GmailSearch(api_resource=<googleapiclient.discovery.Resource object at 0x15df74aa0>),
 GmailGetMessage(api_resource=<googleapiclient.discovery.Resource object at 0x15df74aa0>),
 GmailGetThread(api_resource=<googleapiclient.discovery.Resource object at 0x15df74aa0>)]

In [8]:
pip install -qU "langchain[openai]"

Note: you may need to restart the kernel to use updated packages.


In [11]:
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [13]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

In [14]:
example_query = "Draft an email to fake@fake.com thanking them for coffee."

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Draft an email to fake@fake.com thanking them for coffee.
================================== Ai Message ==================================
Tool Calls:
  create_gmail_draft (call_9a6JXLv9CH5GIDnZcDEGKWx9)
 Call ID: call_9a6JXLv9CH5GIDnZcDEGKWx9
  Args:
    message: Hi,

I just wanted to take a moment to thank you for the coffee we shared the other day. It was a great way to catch up, and I really appreciated the time and conversation.

Looking forward to our next meet-up!

Best regards,

[Your Name]
    to: ['fake@fake.com']
    subject: Thank You for Coffee!
================================= Tool Message =================================
Name: create_gmail_draft

Error: Exception('An error occurred: <HttpError 403 when requesting https://gmail.googleapis.com/gmail/v1/users/me/drafts?alt=json returned "Gmail API has not been used in project 560333382753 before or it is disabled. Enable it by visiting https